In [1]:
atom_elem_to_num = {"H": 1, "C": 6, "O": 8, "Al": 13, "Pt": 78}


def write_opt_file(atom_order, lammps_loc):
    # opt.py file

    with open("opt.py", "w") as f:
        f.write("import re\n")
        f.write("import os\n")
        f.write("import glob\n")
        f.write("\n")
        f.write("from ase.io import *\n")
        f.write("from ase.io.trajectory import TrajectoryWriter\n")
        f.write("import numpy as np\n")
        f.write(
            "from ase.calculators.singlepoint import SinglePointCalculator as SPC\n"
        )
        f.write("from ase.constraints import FixAtoms\n")
        f.write("from pymatgen.io.lammps.data import LammpsData\n")
        f.write("from pymatgen.io.ase import AseAtomsAdaptor\n")
        f.write("def main():\n")
        f.write('    atoms = read("./input.traj")\n')
        f.write("    n = len(atoms)\n")
        f.write("    ase_adap = AseAtomsAdaptor()\n")
        f.write("    atoms_ = ase_adap.get_structure(atoms)\n")
        f.write('    ld = LammpsData.from_structure(atoms_, atom_style="atomic")\n')
        f.write('    ld.write_file("struc.data")\n')
        f.write("\n")
        f.write('    os.system("{} < in.opt")\n'.format(lammps_loc))
        f.write("\n")
        f.write('    images = read("md.lammpstrj", ":")\n')
        f.write('    traj = TrajectoryWriter("opt.traj", "a")\n')
        f.write("\n")
        f.write('    file_name = glob.glob("log.lammps")[0]\n')
        f.write("    f = open(file_name, 'r')\n")
        f.write("    Lines = f.readlines()\n")
        f.write('    patten = r"(\\d+\\s+\\-+\\d*\\.?\\d+)"\n')
        f.write("    e_pot = []\n")
        f.write("    for i, line in enumerate(Lines):\n")
        f.write("        s = line.strip()\n")
        f.write("        match = re.match(patten, s)\n")
        f.write("        if match != None:\n")
        f.write("            D = np.fromstring(s, sep=' ')\n")
        f.write("            e_pot.append(D[1])\n")
        f.write("\n")
        f.write("    print(len(e_pot))\n")
        f.write("\n")
        f.write("    f_all = []\n")
        f.write("    for atoms in images:\n")
        f.write("        f = atoms.get_forces()\n")
        f.write("        f_all.append(f)\n")
        f.write("\n")
        f.write("    for i, atoms in enumerate(images):\n")
        f.write("        an = atoms.get_atomic_numbers()\n")
        for ind, atom in enumerate(atom_order):
            f.write(
                f"        an = [{atom_elem_to_num[atom]} if x == {ind} else x for x in an]\n"
            )
        f.write("\n")
        f.write("        atoms.set_atomic_numbers(an)\n")
        f.write("        traj.write(atoms, energy=e_pot[i], forces=f_all[i])\n")
        f.write("\n")
        f.write('    atoms = read("opt.traj@-1")\n')
        f.write("    e = atoms.get_potential_energy()\n")
        f.write("    f = atoms.get_forces()\n")
        f.write("    pos = atoms.get_positions()\n")
        f.write("    posz = pos[:, 2]\n")
        f.write("    ndx = np.where(posz < 5.5)[0]\n")
        f.write("    c = FixAtoms(ndx)\n")
        f.write("    atoms.set_constraint(c)\n")
        f.write("    atoms.set_calculator(SPC(atoms, energy=e, forces=f))\n")
        f.write('    atoms.write("optimized.traj")\n')


def write_lammps_input_file(model_path, atom_order):
    """
    write lammps input file

    #input
    units                  metal
    dimension	       3
    processors	       * * *
    box tilt 	       large
    boundary 	       p p f

    #real data
    atom_style	       atomic
    read_data	       struc.data

    #potential
    pair_style	allegro
    pair_coeff	* * {model_path} {atom_order}

    timestep 0.0001

    region slab block EDGE EDGE EDGE EDGE 0 5.5
    group fixed_slab region slab
    fix freeze fixed_slab setforce 0.0 0.0 0.0
    dump 1 all custom 1 md.lammpstrj id type x y z fx fy fz
    thermo 1
    thermo_style custom step pe fmax

    #minimize
    min_modify norm max
    minimize 0.0 0.3 200 100000

    """
    with open("in.opt", "w") as f:
        f.write("#input \n")
        f.write("units                  metal\n")
        f.write("dimension	       3\n")
        f.write("processors	       * * *\n")
        f.write("box tilt 	       large\n")
        f.write("boundary 	       p p f\n")
        f.write("\n")
        f.write("#real data\n")
        f.write("atom_style	       atomic\n")
        f.write("read_data	       struc.data\n")
        f.write("\n")
        f.write("#potential\n")
        f.write("pair_style	allegro\n")
        atom_order_str = " ".join(atom_order)
        f.write(f"pair_coeff	* * {model_path} {atom_order_str}\n")
        f.write("\n")
        f.write("timestep 0.0001\n")
        f.write("\n")
        f.write("region slab block EDGE EDGE EDGE EDGE 0 5.5\n")
        f.write("group fixed_slab region slab\n")
        f.write("fix freeze fixed_slab setforce 0.0 0.0 0.0\n")
        f.write("dump 1 all custom 1 md.lammpstrj id type x y z fx fy fz\n")
        f.write("thermo 1\n")
        f.write("thermo_style custom step pe fmax\n")
        f.write("\n")
        f.write("#minimize\n")
        f.write("min_modify norm max\n")
        f.write("minimize 0.0 0.3 200 100000\n")

In [2]:
atom_order = ["Al", "H", "Pt", "C", "O"]
model_path = "mdbh_1.pth"
lammps_loc = "/home/leila/Downloads/lammps-29Oct20/src/lmp_serial"

write_opt_file(atom_order, lammps_loc)
write_lammps_input_file(model_path, atom_order)